# Teil 1, Demo 1: Cholera-Daten von John Snow

## Bibliotheken & Konfiguration

In [ ]:
library(readr)
library(tidyr)
library(ggplot2)
library(leaflet)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-2021-Block-7/master/Rohdaten/john-snow/"

## Zeitverlauf Infektionen & Todesfälle laden

In [ ]:
snow.dates.raw <- read_csv(paste(base_url, "snow.dates.csv", sep=""), 
                           col_names = c("index", "date", "attacks", "deaths"),
                           col_types = cols("i", col_date(format = "%Y-%m-%d"), "i", "i"),
                           skip = 1)
head(snow.dates.raw)

## Zeitverlauf plotten

In [ ]:
snow.dates.long <- snow.dates.raw %>% pivot_longer(!c(index, date), names_to="variable", values_to="n")
options(repr.plot.width = 10, repr.plot.height = 5)
ggplot(snow.dates.long) +
    geom_area(aes(x=date, y=n, fill=variable), alpha=0.5, position="identity") 

## Karte von London laden

In [ ]:
map <- leaflet() %>%
    setView(-0.136722, 51.513578, zoom=16) %>%
    addProviderTiles(providers$Stamen.TonerLite)
map

## Koordinaten der Todesfälle laden

In [ ]:
snow.deaths.raw <- read_csv(paste(base_url, "cholera_deaths.csv", sep=""), 
                            col_names=c("index", "deaths", "lon", "lat"), 
                            col_types = cols("i","i","d","d"), 
                            skip=1)
head(snow.deaths.raw)

## Todesfälle auf der Karte anzeigen

In [ ]:
map <- map %>% addCircleMarkers(
    data = snow.deaths.raw, ~lon, ~lat,
    radius = ~deaths * 2,
    color = "red",
    stroke = FALSE, 
    fillOpacity = 0.5
  )
map

## Koordinaten der Wasserpumpen laden

In [ ]:
snow.pumps.raw <- read_csv(paste(base_url, "johnsnow_pumps.csv", sep=""), 
                           col_names=c("index", "lon", "lat"), 
                           col_types = cols("i","d","d"), 
                           skip=1)
head(snow.pumps.raw)

## Wasserpumpen auf der Karte anzeigen

In [ ]:
map <- map %>% addMarkers(
    data = snow.pumps.raw, ~lon, ~lat,
  )
map